## Управление роботом SCARA

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
from IPython.display import HTML
%matplotlib notebook

In [ ]:
from kinematics import Vector, Quaternion, Transform
import graphics

Возьмем готовые функции для моделирования кинематики:

In [ ]:
scara_l = [220.2, 200, 250]

In [ ]:
def scara_chain(q, l):
    base = Transform.identity()
    column = base + Transform(
        Vector(0, 0, l[0]),
        Quaternion.from_angle_axis(q[0], Vector(0, 0, 1))
    )
    elbow = column + Transform(
        Vector(l[1], 0, 0),
        Quaternion.from_angle_axis(q[1], Vector(0, 0, 1))
    )
    tool = elbow + Transform(
        Vector(l[2], 0, 0),
        Quaternion.from_angle_axis(q[2], Vector(0, 0, 1))
    )
    flange = tool + Transform(
        Vector(0, 0, -q[3]),
        Quaternion.identity()
    )
    return [
        base,
        column,
        elbow,
        tool,
        flange
    ]

Воспользуйтесь своим решением обратной задачи кинематики

In [ ]:
def wrap_from_to(value, s, e):
    r = e - s
    return value - (r * np.floor((value - s) / r))

def scara_ik(target, l):
    d = (target.translation.x ** 2 + target.translation.y ** 2) ** 0.5
    q0 = Vector(1, 0, 0).angle_to(
        Vector(target.translation.x, target.translation.y, 0),
        Vector(0, 0, 1)
    )
    q1 = np.pi / 2
    q2 = 0
    q3 = 100
    return (
        wrap_from_to(q0, -np.pi, np.pi),
        wrap_from_to(q1, -np.pi, np.pi),
        wrap_from_to(q2, -np.pi, np.pi),
        q3
    )

Добавим функцию учитывающую ограничения степеней подвижности:

In [ ]:
scara_lim = [
    (-140, 140),
    (-150, 150),
    (-400, 400),
    (0, 180)
]

In [ ]:
def scara_ik_lim(target, l):
    solution = scara_ik(target, l)
    for index in range(len(solution) - 1):
        if solution[index] < np.deg2rad(scara_lim[index][0]) or\
            solution[index] > np.deg2rad(scara_lim[index][1]) or\
            np.isnan(solution[index]):
            return None
    return solution

Для описания целевого положения будем использовать вектор и угол поворота вокруг вертикальной оси:

In [ ]:
class Target:
    def __init__(self, translation, angle):
        super(Target, self).__init__()
        self.translation = translation
        self.angle = angle
    
    def to_transform(self):
        return Transform(
            self.translation,
            Quaternion.from_angle_axis(
                self.angle,
                Vector(0, 0, 1)
            )
        )

Объявим функцию для описания линейной траектории:

In [ ]:
def lin(start, end, t, total):
    progress = t / total
    return Target(
        Vector.lerp(start.translation, end.translation, progress),
        start.angle + (end.angle - start.angle) * progress
    )

In [ ]:
s = Target(
    Vector(200, 300, 120),
    0
)
e = Target(
    Vector(200, -200, 200),
    np.pi / 2
)

In [ ]:
(x, y, z) = graphics.chain_to_points(
    scara_chain([0, 0, 0, 0], scara_l)
)
fig, ax = graphics.figure(1000)
lines, = ax.plot(x, y, z, color="#000000")
graphics.axis(ax, s.to_transform(), 100)
graphics.axis(ax, e.to_transform(), 100)
r, g, b = graphics.axis(ax, Transform.identity(), 1)

total = 100

def animate(frame):
    trs = lin(s, e, frame, total)
    q = scara_ik_lim(
        trs,
        scara_l
    )
    if q != None:
        chain = scara_chain(q, scara_l)
        (x, y, z) = graphics.chain_to_points(chain)
        lines.set_data_3d(x, y, z)
        global r, g, b
        r.remove(); g.remove(); b.remove()
        r, g, b = graphics.axis(ax, chain[-1], 100)


animate(0)
fps = 25
scara_ani = animation.FuncAnimation(
    fig,
    animate,
    frames=total,
    interval=1000.0/fps
)

In [ ]:
HTML(scara_ani.to_jshtml())

In [ ]:
(x, y, z) = graphics.chain_to_points(
    scara_chain([0, 0, 0, 0, 0, 0], scara_l)
)
fig, ax = graphics.figure(1000)
lines, = ax.plot(x, y, z, color="#000000")
graphics.axis(ax, s.to_transform(), 100)
graphics.axis(ax, e.to_transform(), 100)

total = 100

s_q = scara_ik_lim(s, scara_l)
e_q = scara_ik_lim(e, scara_l)

def animate(frame):
    q = []
    for index in range(len(s_q)):
        t = frame / total
        q += [s_q[index] + t * (e_q[index] - s_q[index])]
    chain = scara_chain(q, scara_l)
    (x, y, z) = graphics.chain_to_points(chain)
    lines.set_data_3d(x, y, z)


animate(0)
fps = 25
scara_ani = animation.FuncAnimation(
    fig,
    animate,
    frames=total,
    interval=1000.0/fps
)

In [ ]:
HTML(scara_ani.to_jshtml())

Опишите разницу между линейным движнием и переброской:
- с точки зрения алгоритма управления
- с точки зрения исполняемого движения
- с точки зрения изменения обобщенных координат (постройте графики)

Напишем функцию для объединения двух линейных движений:

In [ ]:
def lin_lin(start, inter, end, t, total):
    progress = t / total
    if progress < 0.5:
        return lin(start, inter, progress * 2, 1)
    else:
        return lin(inter, end, (progress - 0.5) * 2, 1)

Добавим промежуточную точку:

In [ ]:
i = Target(
    Vector(400, 100, 0),
    np.pi,
)

Рассмотрим два линейных движения в цепочке:

In [ ]:
(x, y, z) = graphics.chain_to_points(
    scara_chain([0, 0, 0, 0, 0, 0], scara_l)
)
fig, ax = graphics.figure(1000)
lines, = ax.plot(x, y, z, color="#000000")
graphics.axis(ax, s.to_transform(), 100)
graphics.axis(ax, i.to_transform(), 100)
graphics.axis(ax, e.to_transform(), 100)

total = 100

def animate(frame):
    trs = lin_lin(s, i, e, frame, total)
    q = scara_ik_lim(
        trs,
        scara_l
    )
    if q != None:
        chain = scara_chain(q, scara_l)
        (x, y, z) = graphics.chain_to_points(chain)
        lines.set_data_3d(x, y, z)


animate(0)
fps = 25
scara_ani = animation.FuncAnimation(
    fig,
    animate,
    frames=total,
    interval=1000.0/fps
)

In [ ]:
HTML(scara_ani.to_jshtml())

Проанализируем скорости обобщенных координат:

In [ ]:
v_lin_lin = np.vectorize(lin_lin, excluded={0, 1, 2, 4})
v_irb_ik = np.vectorize(scara_ik_lim, excluded={1, 2})
total = 20
step = 0.01
t = np.arange(0, total, step)

fig = plt.figure()
ax = fig.add_subplot()
w = np.diff(v_irb_ik(
    v_lin_lin(s, i, e, t, total),
    scara_l,
)) / step;
ax.plot(t[:-1], w[0], label="$\omega_0$")
ax.plot(t[:-1], w[1], label="$\omega_1$")
ax.plot(t[:-1], w[2], label="$\omega_2$")
ax.plot(t[:-1], w[3], label="$\omega_3$")
fig.legend()
fig.show()

Как меняется скорость приводов?

Напишем функцию для объединения двух линейных движений со сглаживанем:

In [ ]:
def bezier_target(a, b, c, t):
    position = Vector.lerp(
        Vector.lerp(a.translation, b.translation, t),
        Vector.lerp(b.translation, c.translation, t),
        t
    )
    rotation = (1 - t) ** 2 * a.angle +\
        2 * t * (1 - t) * b.angle +\
        t**2 * c.angle
    return Target(position, rotation)

def lin_lin_smooth(start, inter, end, t, total, blend=0.1):
    progress = t / total
    if np.abs(progress - 0.5) < blend:
        progress = (progress - 0.5 + blend) / 2 / blend
        a = lin(start, inter, 1.0 - 2 * blend, 1)
        b = inter
        c = lin(inter, end, 2 * blend, 1)
        return bezier_target(
            a,
            b,
            c,
            progress
        )
    else:
        return lin_lin(start, inter, end, t, total)

In [ ]:
blending = 0.1

In [ ]:
(x, y, z) = graphics.chain_to_points(
    scara_chain([0, 0, 0, 0, 0, 0], scara_l)
)
fig, ax = graphics.figure(1000)
lines, = ax.plot(x, y, z, color="#000000")
graphics.axis(ax, s.to_transform(), 100)
graphics.axis(ax, i.to_transform(), 100)
graphics.axis(ax, e.to_transform(), 100)

total = 100

def animate(frame):
    trs = lin_lin_smooth(s, i, e, frame, total, 0.1)
    q = scara_ik_lim(
        trs,
        scara_l,
    )
    if q != None:
        chain = scara_chain(q, scara_l)
        (x, y, z) = graphics.chain_to_points(chain)
        lines.set_data_3d(x, y, z)


animate(0)
fps = 25
scara_ani = animation.FuncAnimation(
    fig,
    animate,
    frames=total,
    interval=1000.0/fps
)

Снова проанализируем скорости обобщенных координат:

In [ ]:
v_lin_lin = np.vectorize(lin_lin_smooth, excluded={0, 1, 2, 4, 5})
v_irb_ik = np.vectorize(scara_ik_lim, excluded={1, 2})
total = 20
step = 0.01
t = np.arange(0, total, step)

fig = plt.figure()
ax = fig.add_subplot()
w = np.diff(v_irb_ik(
    v_lin_lin(s, i, e, t, total, blending),
    scara_l,
)) / step;
ax.plot(t[:-1], w[0], label="$\omega_0$")
ax.plot(t[:-1], w[1], label="$\omega_1$")
ax.plot(t[:-1], w[2], label="$\omega_2$")
ax.plot(t[:-1], w[3], label="$\omega_3$")
fig.legend()
fig.show()

Как меняется скорость приводов?

## Самостоятельное задание:
Проведите исследование для своих точек траектории.

Проанализируйте влияние параметра `blend` на скорость обобщенных координат.

Оцените ускорения обобщенных координат (через дифференцирование второго порядка `diff(..., 2`).